In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time

# Import API key
# import api_keys

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

583

In [7]:
# make the dataframe
city_df = pd.DataFrame({"city":cities})

## Perform API Calls

In [13]:
# OpenWeatherMap API Key
api_key = Your Key here

# Starting URL for Weather Map API Call
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + api_key 

# set up functino to grab required values
def grab_values(city, url=url):
    city_url = url + "&q=" + city
    response_json = requests.get(city_url).json()
    
    try:
        latitude = response_json["coord"]["lat"]
        longitude = response_json["coord"]["lon"]
        temperature = response_json["main"]["temp"]
        humidity = response_json["main"]["humidity"]
        cloudiness = response_json["clouds"]["all"]
        wind_speed = response_json["wind"]["speed"]

    except KeyError as E:
        print(f"Key error incurred on key: {E} while parsing json response from {city}")
        latitude = np.nan
        longitude = np.nan
        temperature = np.nan
        humidity = np.nan
        cloudiness = np.nan
        wind_speed = np.nan
        
    return pd.Series([city, latitude, longitude, temperature, humidity, cloudiness, wind_speed], index=["city", "latitude", "longitude", "temperature", "humidity", "cloudiness", "wind_speed"])

In [14]:
combined = pd.merge(
    city_df.iloc[:5], 
    city_df.loc[:5,'city'].apply(grab_values),
    on='city')\
    .dropna(axis=0,how='any')
combined

,city,latitude,longitude,temperature,humidity,cloudiness,wind_speed
0,sevsk,52.15,34.49,80.67,43,0,5.39
1,chuy,-33.69,-53.46,58.35,85,92,22.73
2,villarrica,-39.28,-72.23,46.47,64,80,2.93
3,mahebourg,-20.41,57.70,73.40,73,40,12.75
4,albany,42.65,-73.75,62.56,75,90,3.36
